In [1]:
# from wandbhelper.util import init_wandb, log_all_plots, log_plot_as_image

# wandb_run_id = init_wandb(run_name="price_match_simulation")
# print(f"Run ID {wandb_run_id}")

In [2]:
import pandas as pd
import pickle
import time

from ml_simulation.dataset_split import customer_split
from ml_simulation.util import HiddenPrints
from ml_features.features import create_features
from ml_training.train_xgb import train_xgb
from ml_simulation__budget.sample import sample_budget_alternative_customers
from ml_simulation__budget.data import get_budget_alternative_compute_function
from ml_simulation__budget.widget import show_budget_alternative_widget
    
import warnings
warnings.filterwarnings('ignore')

df_quotes = pd.read_csv('cleaned_quote_data.csv')
df_quotes['dt_creation_devis'] = pd.to_datetime(df_quotes['dt_creation_devis'])

In [3]:
split_result = customer_split(df_quotes)
df_train = split_result['train']
df_sim = split_result['simulation']


SPLIT CUSTOMERS: TRAIN vs SIMULATION: TRAINING SIZE 0.95
Split: 22708 train, 1180 sim customers


In [4]:
TRAIN = False

In [5]:
# Model building
if TRAIN:
    with HiddenPrints():
        X_train = create_features(df_train)
    
    y_train = X_train['converted']
    X_train = X_train.drop(['numero_compte', 'converted'], axis=1)
    feature_names = X_train.columns.tolist()
    
    result = train_xgb(X_train, y_train, "simulation_poc")
    model = result['model']
    feature_names = result['features']
    
    print(f"Model trained: {len(feature_names)} features")
else:
    with open('simulation_poc.pkl', 'rb') as file:
        model_data = pickle.load(file)
        model = model_data['model']
        feature_names = model_data['features']

In [6]:
# Sampling
sample_seed = int(time.time() * 1000) % 10000000
selected_ids = sample_budget_alternative_customers(df_sim, random_state=sample_seed)

Non-converted customers: 923
Products with price data: 9

📊 PRODUCT PRICE TIERS (from simulation pool):
                         product  count       p30       p70       p90
                       Chaudière    411  3646.800  5398.340  7688.060
                           Poêle    292  4938.600  6300.307  7573.243
                 Pompe à chaleur    197 12546.528 15726.256 18570.966
                   Climatisation    191  3816.500  6849.280 11113.260
ECS : Chauffe-eau ou adoucisseur     46  1380.105  2830.515  3603.930
                  Photovoltaïque     35  7565.466 10289.626 13560.080
                          Autres      9  1330.800  6100.760  7918.098
                     Produit VMC      8  2378.767  5365.288 19652.460
                Appareil hybride      7 14456.038 14773.304 15262.136

Premium product candidates: 265

🎯 SELECTED BUDGET ALTERNATIVE CANDIDATES:
customer_id                          product    price    tier  budget_price
 CL00171305 ECS : Chauffe-eau ou adoucisseur

In [7]:
# Simulation
compute = get_budget_alternative_compute_function(model, feature_names, df_sim, selected_ids)
show_budget_alternative_widget(compute, selected_ids)

In [8]:
# SCENARIO 6: BUDGET ALTERNATIVE - SUMMARY TABLE
print("\n=== 📊 SCENARIO 6: BUDGET ALTERNATIVE RESULTS ===\n")

import numpy as np
import pandas as pd
from ml_inference.inference import safe_predict

# ============================================
# 1. USE YOUR ACTUAL SELECTED CUSTOMERS
# ============================================
# These come from your sampler - no hardcoding!
print(f"Analyzing {len(selected_ids)} premium customers: {selected_ids}")

# ============================================
# 2. GET BUDGET PRICES FROM THE DATA
# ============================================
# Calculate price tiers from simulation pool
df_all = df_sim.copy()
product_tiers = {}

for product in df_all['famille_equipement_produit'].unique():
    prices = df_all[df_all['famille_equipement_produit'] == product]['mt_apres_remise_ht_devis'].dropna()
    if len(prices) >= 5:
        p30 = prices.quantile(0.3)
        product_tiers[product] = p30

print(f"📊 Product tiers loaded for {len(product_tiers)} products")

# ============================================
# 3. COLLECT CUSTOMER DATA
# ============================================
customers = []
missing_tiers = []

for cust_id in selected_ids:
    quotes = df_sim[df_sim['numero_compte'] == cust_id].copy()
    if len(quotes) == 0:
        print(f"⚠️ No quotes found for {cust_id}")
        continue
        
    baseline = safe_predict(cust_id, quotes, model, feature_names)
    product = quotes['famille_equipement_produit'].iloc[0]
    price = quotes['mt_apres_remise_ht_devis'].sum()
    
    # Get budget price from product tiers
    if product in product_tiers:
        budget_price = product_tiers[product]
    else:
        # Fallback: 30% reduction
        budget_price = price * 0.7
        missing_tiers.append(product)
    
    # Simulate budget version
    mod = quotes.copy()
    mod['mt_apres_remise_ht_devis'] = budget_price
    budget_prob = safe_predict(cust_id, mod, model, feature_names)
    delta = budget_prob - baseline
    reduction = (1 - budget_price/price) * 100
    
    customers.append({
        'id': cust_id,
        'product': product,
        'price': price,
        'budget_price': budget_price,
        'reduction': reduction,
        'baseline': baseline,
        'budget_prob': budget_prob,
        'delta': delta
    })

if missing_tiers:
    print(f"⚠️ Missing tiers for: {set(missing_tiers)} (used 30% fallback)")

# ============================================
# 4. CREATE SUMMARY TABLE
# ============================================
results = []
for cust in customers:
    # Determine verdict
    if cust['delta'] > 0.01:
        verdict = "✅ FORTE AMÉLIORATION"
    elif cust['delta'] > 0:
        verdict = "✅ Légère amélioration"
    elif cust['delta'] > -0.01:
        verdict = "⚪ Neutre"
    else:
        verdict = "❌ DÉTÉRIORATION"
    
    results.append({
        'Client': cust['id'][:10],
        'Produit': cust['product'][:30],
        'Prix actuel': f"€{cust['price']:,.0f}",
        'Prix budget': f"€{cust['budget_price']:,.0f}",
        'Réduction': f"{cust['reduction']:.0f}%",
        'Probabilité actuelle': f"{cust['baseline']:.3f}",
        'Probabilité budget': f"{cust['budget_prob']:.3f}",
        'Δ': f"{cust['delta']:+.3f}",
        'Verdict': verdict
    })

if not results:
    print("❌ No results to display!")
else:
    df_results = pd.DataFrame(results)

    # ============================================
    # 5. DISPLAY TABLE
    # ============================================
    print("\n📋 BUDGET ALTERNATIVE - RÉSULTATS PAR CLIENT")
    print("=" * 140)
    print(df_results.to_string(index=False))
    print("=" * 140)

    # ============================================
    # 6. CALCULATE AGGREGATE STATISTICS
    # ============================================
    baselines = [c['baseline'] for c in customers]
    budget_probs = [c['budget_prob'] for c in customers]
    deltas = [c['delta'] for c in customers]
    reductions = [c['reduction'] for c in customers]

    print("\n📈 STATISTIQUES GLOBALES:")
    print(f"   Probabilité moyenne actuelle: {np.mean(baselines):.3f}")
    print(f"   Probabilité moyenne budget:   {np.mean(budget_probs):.3f}")
    print(f"   Δ moyen:                      {np.mean(deltas):+.3f}")
    print(f"   Réduction moyenne:             {np.mean(reductions):.0f}%")
    print(f"   Réduction médiane:             {np.median(reductions):.0f}%")

    # ============================================
    # 7. COUNT IMPROVEMENTS
    # ============================================
    improved = sum(1 for d in deltas if d > 0.01)
    slight = sum(1 for d in deltas if 0 < d <= 0.01)
    neutral = sum(1 for d in deltas if -0.01 <= d <= 0)
    worse = sum(1 for d in deltas if d < -0.01)

    print("\n📊 RÉPARTITION DES RÉSULTATS:")
    print(f"   ✅ Forte amélioration (>+0.01): {improved}/{len(customers)}")
    print(f"   📈 Légère amélioration (0 à +0.01): {slight}/{len(customers)}")
    print(f"   ⚪ Neutre (-0.01 à 0): {neutral}/{len(customers)}")
    print(f"   ❌ Détérioration (<-0.01): {worse}/{len(customers)}")

    # ============================================
    # 8. PRODUCT-SPECIFIC INSIGHTS
    # ============================================
    print("\n🔍 INSIGHTS PAR PRODUIT:")
    for cust in customers:
        if cust['delta'] > 0.01:
            emoji = "✅"
        elif cust['delta'] > 0:
            emoji = "📈"
        elif cust['delta'] > -0.01:
            emoji = "⚪"
        else:
            emoji = "❌"
        
        print(f"   {emoji} {cust['product'][:25]}: {cust['delta']:+.3f} (réduction {cust['reduction']:.0f}%)")

    # ============================================
    # 9. KEY INSIGHTS
    # ============================================
    print("\n💡 INSIGHTS CLÉS:")
    
    if improved > len(customers)/2:
        print("   ✅ La version budget fonctionne pour la PLUPART des produits")
    elif improved + slight > len(customers)/2:
        print("   📈 La version budget aide certains produits")
    else:
        print("   ⚠️ La version budget n'est pas systématiquement bénéfique")
        
    print(f"   📊 Réduction moyenne de {np.mean(reductions):.0f}% sur les prix")
    
    # Best and worst products
    best = max(customers, key=lambda x: x['delta'])
    worst = min(customers, key=lambda x: x['delta'])
    print(f"   ✅ Meilleur: {best['product'][:20]} ({best['delta']:+.3f})")
    print(f"   ❌ Pire: {worst['product'][:20]} ({worst['delta']:+.3f})")

    # ============================================
    # 10. EXPORT OPTION (uncomment to save)
    # ============================================
    # df_results.to_csv('budget_alternative_results.csv', index=False)
    # print("\n✅ Résultats exportés vers budget_alternative_results.csv")


=== 📊 SCENARIO 6: BUDGET ALTERNATIVE RESULTS ===

Analyzing 5 premium customers: ['CL00171305', 'CL00066986', 'CL00142595', 'CL00067051', 'CL00066737']
📊 Product tiers loaded for 11 products

📋 BUDGET ALTERNATIVE - RÉSULTATS PAR CLIENT
    Client                        Produit Prix actuel Prix budget Réduction Probabilité actuelle Probabilité budget      Δ               Verdict
CL00171305 ECS : Chauffe-eau ou adoucisse      €3,541      €1,371       61%                0.476              0.476 +0.000              ⚪ Neutre
CL00066986                          Poêle      €8,325      €4,862       42%                0.228              0.224 -0.003              ⚪ Neutre
CL00142595                    Produit VMC      €5,416      €2,537       53%                0.340              0.344 +0.004 ✅ Légère amélioration
CL00067051                         Autres      €9,022      €1,363       85%                0.293              0.310 +0.017  ✅ FORTE AMÉLIORATION
CL00066737                Pompe à chal